<a href="https://colab.research.google.com/github/Dhananjay42/cs6910-assn3/blob/main/assn3_nb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##CS6910: Assignment-3
-- Submitted by Dhananjay Balakrishnan, ME19B012

##Setup and Loading the Dataset

In [1]:
!pip install python-Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/174.1 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 106.5 MB/s eta 0:00:00


In [2]:
import csv
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import Levenshtein
import math

In [3]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [4]:
data_dir = '/content/gdrive/MyDrive/CS6910_A3/aksharantar_sampled/tam/'

In [5]:
def obtain_data(dir):
  x = []
  y = []

  with open(dir, 'r') as file:
    reader = csv.DictReader(file, fieldnames=['x', 'y'])
  
    for row in reader:
      x.append(row['x'])
      y.append(row['y'])
    
  return x, y

In [6]:
x_train, y_train = obtain_data(data_dir + 'tam_train.csv')
x_test, y_test = obtain_data(data_dir + 'tam_test.csv')
x_val, y_val = obtain_data(data_dir + 'tam_valid.csv')

In [7]:
for i in range(0, 2):
  print(x_train[i], y_train[i])

thottacharya தொட்டாச்சார்ய
menmaithaan மென்மைதான்


In [8]:
start_token = 0
end_token = 1

In [9]:
eng_characters = {}

In [10]:
class Language:
  def __init__(self, name):
    self.name = name
    self.char2index = {}
    self.index2char = {0: "SOS", 1: "EOS", 2: "unknown"}
    self.n_chars = 3  # Count SOS and EOS
    self.max_size = 2

  def update_vocab(self, x):
    for word in x:
      if len(word) + 2 > self.max_size:
        self.max_size = len(word) + 2

      for letter in word:
        if letter not in self.char2index.keys():
          self.char2index[letter] = self.n_chars
          self.index2char[self.n_chars] = letter
          self.n_chars = self.n_chars + 1
  
  def get_index(self, character):
    if character in self.char2index.keys():
      return self.char2index[character]
    else:
      return 2
  
  def get_character(self, index):
    if index == 2:
      return '$'
    else:
      return self.index2char[index]

In [11]:
english = Language('eng')
tamil = Language('tam')

In [12]:
english.update_vocab(x_train)
tamil.update_vocab(y_train)

In [13]:
def encoded_word(language, word):
  coded = [language.get_index(letter) for letter in word]
  coded.append(end_token)
  return coded

def get_pairs(lang1, lang2, inputs, targets):
  return [(torch.tensor(encoded_word(lang1, x), dtype=torch.long, device=device).view(-1, 1), torch.tensor(encoded_word(lang2, y), dtype=torch.long, device=device).view(-1, 1)) 
  for (x,y) in zip(inputs,targets)]

def decoded_word(language, encoded_word):
  characters = [language.get_character(num) for num in encoded_word[:-1]]
  decoded = ''.join(characters)
  return decoded

In [14]:
train_data = get_pairs(english, tamil, x_train, y_train)
test_data = get_pairs(english, tamil, x_test, y_test)
val_data = get_pairs(english, tamil, x_val, y_val)

#RNN without Attention

In [15]:
class Encoder(nn.Module):
  def __init__(self, inp_vocab_size, embedding_size, n_layers, hl_size, dropout, cell_type, bidirectional):
    super(Encoder, self).__init__()
    self.vocab_size = inp_vocab_size
    self.embedding_size = embedding_size
    self.n_layers = n_layers
    self.hl_size = hl_size
    self.bidirectional = bidirectional
    self.cell_type = cell_type
    self.dropout = dropout

    if cell_type == 'RNN':
      self.cell = nn.RNN(self.embedding_size, self.hl_size, num_layers = self.n_layers, dropout = self.dropout, bidirectional = self.bidirectional).to(device)
    elif cell_type == 'GRU':
      self.cell = nn.GRU(self.embedding_size, self.hl_size, num_layers = self.n_layers, dropout = self.dropout, bidirectional = self.bidirectional).to(device)
    elif cell_type == 'LSTM':
      self.cell = nn.LSTM(self.embedding_size, self.hl_size, num_layers = self.n_layers, dropout = self.dropout, bidirectional = self.bidirectional).to(device)
    else:
      print('Wrong Cell Type.')
      exit()
    self.embedding_layer = nn.Embedding(self.vocab_size, self.embedding_size).to(device)
  
  def forward(self, input, hidden, c = 0):
    embedded = self.embedding_layer(input).view(1, 1, -1)
    if self.cell_type != 'LSTM':
      output, hidden = self.cell(embedded, hidden)
      
      return output, hidden

    else:
      output, (hidden, c) = self.cell(embedded, (hidden, c))

      return output, hidden, c
  
  def init_hidden(self):
    if self.bidirectional:
      return torch.zeros(self.n_layers*2, 1, self.hl_size, device = device)
    else:
      return torch.zeros(self.n_layers, 1, self.hl_size, device = device)

In [16]:
class DecoderVanilla(nn.Module):
  def __init__(self, out_vocab_size, embedding_size, n_layers, hl_size, dropout, cell_type, bidirectional):
    super(DecoderVanilla, self).__init__()
    self.vocab_size = out_vocab_size
    self.embedding_size = embedding_size
    self.n_layers = n_layers
    self.hl_size = hl_size
    self.softmax = nn.LogSoftmax(dim=1)
    self.cell_type = cell_type
    self.bidirectional = bidirectional
    self.dropout = dropout

    if self.bidirectional:
      self.linear = nn.Linear(2*self.hl_size, self.vocab_size).to(device)
    else:
      self.linear = nn.Linear(self.hl_size, self.vocab_size).to(device)

    if cell_type == 'RNN':
      self.cell = nn.RNN(self.embedding_size, self.hl_size, num_layers = self.n_layers, dropout = self.dropout, bidirectional = self.bidirectional).to(device)
    elif cell_type == 'GRU':
      self.cell = nn.GRU(self.embedding_size, self.hl_size, num_layers = self.n_layers, dropout = self.dropout, bidirectional = self.bidirectional).to(device)
    elif cell_type == 'LSTM':
      self.cell = nn.LSTM(self.embedding_size, self.hl_size, num_layers = self.n_layers, dropout = self.dropout, bidirectional = self.bidirectional).to(device)
    else:
      print('Wrong Cell Type.')
      exit()
    
    self.embedding_layer = nn.Embedding(self.vocab_size, self.embedding_size).to(device)
  
  def forward(self, input, hidden, c = 0):
    embedded = self.embedding_layer(input).view(1, 1, -1)
    output = F.relu(embedded)

    if self.cell_type != 'LSTM':
      output, hidden = self.cell(output, hidden)
      output = self.linear(output[0])
      output = self.softmax(output)
      return output, hidden
    else:
      output, (hidden, c) = self.cell(output, (hidden, c))
      output = self.linear(output[0])
      output = self.softmax(output)
      return output, hidden, c

In [17]:
class seq2seq_vanilla():
  def __init__(self, inp_language, out_language, embedding_size, n_layers, hl_size, dropout = 0.2, cell_type = 'LSTM', lr = 0.01, teacher_forcing_ratio = 0.5,bidirectional_flag = False):
    self.encoder = Encoder(inp_language.n_chars, embedding_size, n_layers, hl_size, dropout, cell_type, bidirectional = bidirectional_flag)
    self.decoder = DecoderVanilla(out_language.n_chars, embedding_size, n_layers, hl_size, dropout, cell_type, bidirectional = bidirectional_flag)
    self.lr = lr
    self.teacher_forcing = teacher_forcing_ratio
    self.max_length = out_language.max_size
    self.cell_type = cell_type

    self.encoder_optimizer = optim.SGD(self.encoder.parameters(), lr=self.lr)
    self.decoder_optimizer = optim.SGD(self.decoder.parameters(), lr = self.lr)

    self.loss_fn = nn.NLLLoss()

  def train_step(self, input, target):
    encoder_hidden = self.encoder.init_hidden()
    encoder_c = self.encoder.init_hidden()

    self.encoder_optimizer.zero_grad()
    self.decoder_optimizer.zero_grad()

    input_length = input.size(0)
    target_length = target.size(0)

    loss = 0

    for i in range(0, input_length):
      if self.cell_type != 'LSTM':
        encoder_output, encoder_hidden = self.encoder.forward(input[i], encoder_hidden)
      else:
        encoder_output, encoder_hidden, encoder_c = self.encoder.forward(input[i], encoder_hidden, encoder_c)
    
    decoder_input = torch.tensor([[start_token]], device=device)

    decoder_hidden = encoder_hidden
    decoder_c = encoder_c

    num = random.random()

    if num < self.teacher_forcing:
      #here, we use teacher forcing. 
      for j in range(0, target_length):
        if self.cell_type != 'LSTM':
          decoder_output, decoder_hidden = self.decoder.forward(decoder_input, decoder_hidden)
        else:
          decoder_output, decoder_hidden, decoder_c = self.decoder.forward(decoder_input, decoder_hidden, decoder_c)

        loss = loss + self.loss_fn(decoder_output, target[j])
        decoder_input = target[j]#.unsqueeze(0)

    else:
      #here, there is no teacher forcing. the predictions themselves are used. 
      for j in range(0, target_length):
        if self.cell_type != 'LSTM':
          decoder_output, decoder_hidden = self.decoder.forward(decoder_input, decoder_hidden)
        else:
          decoder_output, decoder_hidden, decoder_c = self.decoder.forward(decoder_input, decoder_hidden, decoder_c)

        loss = loss + self.loss_fn(decoder_output, target[j])
        value, index = decoder_output.topk(1)
        decoder_input = index.squeeze().detach()
        if decoder_input.item() == end_token:
          break
      
            
    loss.backward()
    self.encoder_optimizer.step()
    self.decoder_optimizer.step()

    return loss.item()/target_length
  
  def predict(self, input, target):
    with torch.no_grad():
      encoder_hidden = self.encoder.init_hidden()
      encoder_c = self.encoder.init_hidden()

      input_length = input.size(0)
      for i in range(0, input_length):
        if self.cell_type != 'LSTM':
          encoder_output, encoder_hidden = self.encoder.forward(input[i], encoder_hidden)
        else:
          encoder_output, encoder_hidden, encoder_c = self.encoder.forward(input[i], encoder_hidden, encoder_c)

      decoder_input = torch.tensor([[start_token]], device=device)
      decoder_hidden = encoder_hidden
      decoder_c = encoder_c

      outputs = []
      for i in range(0, self.max_length):
        if self.cell_type != 'LSTM':
          decoder_output, decoder_hidden = self.decoder.forward(decoder_input, decoder_hidden)
        else:
          decoder_output, decoder_hidden, decoder_c = self.decoder.forward(decoder_input, decoder_hidden, decoder_c)

        value, index = decoder_output.data.topk(1)
        decoder_input = index.squeeze().detach()
        outputs.append(decoder_input.item())
        if decoder_input.item() == end_token:
          break

      return outputs


  def evaluate(self, data):
    correct = 0
    character_wise = 0
    count = 0
    total_distance = 0

    for pair in data:
      input = pair[0]
      target = pair[1]
      pred = self.predict(input, target)
      target = target.tolist()
      target = [t[0] for t in target]

      #if count%500 == 0:
        #print(decoded_word(tamil,pred), decoded_word(tamil,target))
      #count = count + 1

      pred_word = decoded_word(tamil,pred)
      tar_word = decoded_word(tamil,target)
      
      if pred_word == tar_word:
        correct = correct + 1
      
      total_distance = total_distance + min((Levenshtein.distance(pred_word, tar_word)/max(len(tar_word),len(pred_word))), 1)
    
    avg_distance = total_distance/len(data)
    char_acc = 1 - avg_distance
    acc = correct/len(data)


    return acc, char_acc

In [18]:
model = seq2seq_vanilla(inp_language = english, out_language = tamil, embedding_size = 64, n_layers = 2, hl_size = 128)

In [19]:
n_iters = 75000

In [20]:
print(len(val_data))

4096


In [21]:
training_pairs = [random.choice(train_data) for i in range(0, n_iters)]

In [22]:
# train_loss = 0
# for i in range(0, n_iters):
#   training_pair = training_pairs[i]
#   x = training_pair[0]
#   y = training_pair[1]
#   loss = model.train_step(x, y)
#   train_loss = train_loss + loss

#   if (i+1)%5000 == 0:
#     print('------------------------------------------------')
#     print('train loss is:', train_loss/5000)
#     test_acc, char_acc = model.evaluate(val_data)
#     print(f'test accuracy is {test_acc} and character-wise accuracy is {char_acc}')
#     train_loss = 0

#RNN with Attention

In [49]:
class DecoderAttn(nn.Module):
  def __init__(self, out_vocab_size, max_length, embedding_size, hl_size, dropout, cell_type, bidirectional):
    super(DecoderAttn, self).__init__()
    self.vocab_size = out_vocab_size
    self.embedding_size = embedding_size
    self.hl_size = hl_size
    self.softmax = nn.LogSoftmax(dim=1)
    self.cell_type = cell_type
    self.bidirectional = bidirectional
    self.dropout = dropout
    self.max_length = max_length

    self.dropout_layer = nn.Dropout(self.dropout).to(device)


    if self.bidirectional:
      self.linear = nn.Linear(2*self.hl_size, self.vocab_size).to(device)
      self.attention = nn.Linear(2*self.hl_size + self.embedding_size, self.max_length).to(device)
      self.attn_combine = nn.Linear(2*self.hl_size + self.embedding_size, self.embedding_size).to(device)

    else:
      self.linear = nn.Linear(self.hl_size, self.vocab_size).to(device)
      self.attention = nn.Linear(self.hl_size + self.embedding_size, self.max_length).to(device)
      self.attn_combine = nn.Linear(self.hl_size + self.embedding_size, self.embedding_size).to(device)


    if cell_type == 'RNN':
      self.cell = nn.RNN(self.embedding_size, self.hl_size, num_layers = 1, bidirectional = self.bidirectional).to(device)
    elif cell_type == 'GRU':
      self.cell = nn.GRU(self.embedding_size, self.hl_size, num_layers = 1, bidirectional = self.bidirectional).to(device)
    elif cell_type == 'LSTM':
      self.cell = nn.LSTM(self.embedding_size, self.hl_size, num_layers = 1, bidirectional = self.bidirectional).to(device)
    else:
      print('Wrong Cell Type.')
      exit()
    
    self.embedding_layer = nn.Embedding(self.vocab_size, self.embedding_size).to(device)
  
  def forward(self, input, hidden, encoder_outputs, c = 0):
    embedded = self.embedding_layer(input).view(1, 1, -1)
    embedded = self.dropout_layer(embedded)

    if self.bidirectional:
      concatenated = torch.cat((embedded[0], hidden[0], hidden[1]), 1)
    else:
      concatenated = torch.cat((embedded[0], hidden[0]), 1)
    #print(embedded.shape, hidden.shape, concatenated.shape)
    #print(embedded.shape, hidden.shape, concatenated.shape)

    attn_weights = F.softmax(self.attention(concatenated), dim=1)
    attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))

    output = torch.cat((embedded[0], attn_applied[0]), 1)
    output = self.attn_combine(output).unsqueeze(0)

    if self.cell_type != 'LSTM':
      output, hidden = self.cell(output, hidden)
      output = self.linear(output[0])
      output = self.softmax(output)
      return output, hidden, attn_weights
    else:
      output, (hidden, c) = self.cell(output, (hidden, c))
      output = self.linear(output[0])
      output = self.softmax(output)
      return output, hidden, c, attn_weights

In [53]:
class seq2seq_attn():
  #out_vocab_size, max_length, embedding_size, hl_size, dropout, cell_type, bidirectional
  #self, inp_vocab_size, embedding_size, n_layers, hl_size, dropout, cell_type, bidirectional):
  def __init__(self, inp_language, out_language, embedding_size, hl_size, dropout = 0.2, cell_type = 'LSTM', lr = 0.005, teacher_forcing_ratio = 0.5,bidirectional_flag = True):
    self.max_length = max(out_language.max_size,  inp_language.max_size)
    self.encoder = Encoder(inp_language.n_chars, embedding_size, 1, hl_size, dropout, cell_type, bidirectional = bidirectional_flag)
    self.decoder = DecoderAttn(out_language.n_chars, self.max_length, embedding_size, hl_size, dropout, cell_type, bidirectional = bidirectional_flag)
    self.lr = lr
    self.teacher_forcing = teacher_forcing_ratio
    self.cell_type = cell_type
    self.bidir = bidirectional_flag

    self.encoder_optimizer = optim.SGD(self.encoder.parameters(), lr=self.lr)
    self.decoder_optimizer = optim.SGD(self.decoder.parameters(), lr = self.lr)

    self.loss_fn = nn.NLLLoss()
    #decay^30000 = 0.2
    self.decay_rate = 2**(math.log2(0.2)/30000)
    #self.decay = lambda epoch: self.decay_rate
    self.encoder_scheduler = torch.optim.lr_scheduler.MultiplicativeLR(self.encoder_optimizer, lr_lambda=self.decay)
    self.decoder_scheduler = torch.optim.lr_scheduler.MultiplicativeLR(self.decoder_optimizer, lr_lambda=self.decay)
  
  def decay(self, epoch):
    return self.decay_rate

  def train_step(self, input, target):
    encoder_hidden = self.encoder.init_hidden()
    encoder_c = self.encoder.init_hidden()

    self.encoder_optimizer.zero_grad()
    self.decoder_optimizer.zero_grad()

    input_length = input.size(0)
    target_length = target.size(0)

    loss = 0

    if self.bidir == True:
      encoder_outputs = torch.zeros(self.max_length, 2*self.encoder.hl_size, device=device)
    else:
      encoder_outputs = torch.zeros(self.max_length, self.encoder.hl_size, device=device)

    for i in range(0, input_length):
      if self.cell_type != 'LSTM':
        encoder_output, encoder_hidden = self.encoder.forward(input[i], encoder_hidden)
        encoder_outputs[i] = encoder_output[0, 0]
      else:
        encoder_output, encoder_hidden, encoder_c = self.encoder.forward(input[i], encoder_hidden, encoder_c)
        encoder_outputs[i] = encoder_output[0, 0]
    
    decoder_input = torch.tensor([[start_token]], device=device)

    decoder_hidden = encoder_hidden
    decoder_c = encoder_c

    num = random.random()

    if num < self.teacher_forcing:
      #here, we use teacher forcing. 
      for j in range(0, target_length):
        if self.cell_type != 'LSTM':
          decoder_output, decoder_hidden, decoder_attention = self.decoder.forward(decoder_input, decoder_hidden, encoder_outputs)
        else:
          decoder_output, decoder_hidden, decoder_c, decoder_attention = self.decoder.forward(decoder_input, decoder_hidden, encoder_outputs, decoder_c)

        loss = loss + self.loss_fn(decoder_output, target[j])
        decoder_input = target[j]#.unsqueeze(0)

    else:
      #here, there is no teacher forcing. the predictions themselves are used. 
      for j in range(0, target_length):
        if self.cell_type != 'LSTM':
          decoder_output, decoder_hidden, decoder_attention = self.decoder.forward(decoder_input, decoder_hidden, encoder_outputs)
        else:
          decoder_output, decoder_hidden, decoder_c, decoder_attention = self.decoder.forward(decoder_input, decoder_hidden, encoder_outputs, decoder_c)

        loss = loss + self.loss_fn(decoder_output, target[j])
        value, index = decoder_output.topk(1)
        decoder_input = index.squeeze().detach()
        if decoder_input.item() == end_token:
          break
      
            
    loss.backward()
    self.encoder_optimizer.step()
    self.decoder_optimizer.step()

    self.encoder_scheduler.step()
    self.decoder_scheduler.step()

    return loss.item()/target_length
  
  def predict(self, input, target):
    with torch.no_grad():
      encoder_hidden = self.encoder.init_hidden()
      encoder_c = self.encoder.init_hidden()

      if self.bidir == True:
        encoder_outputs = torch.zeros(self.max_length, 2*self.encoder.hl_size, device=device)
      else:
        encoder_outputs = torch.zeros(self.max_length, self.encoder.hl_size, device=device)

      input_length = input.size(0)
      for i in range(0, min(input_length, self.max_length)):
        if self.cell_type != 'LSTM':
          encoder_output, encoder_hidden = self.encoder.forward(input[i], encoder_hidden)
          encoder_outputs[i] = encoder_output[0, 0]
        else:
          encoder_output, encoder_hidden, encoder_c = self.encoder.forward(input[i], encoder_hidden, encoder_c)
          encoder_outputs[i] = encoder_output[0, 0]

      decoder_input = torch.tensor([[start_token]], device=device)
      decoder_hidden = encoder_hidden
      decoder_c = encoder_c

      outputs = []
      for i in range(0, self.max_length):
        if self.cell_type != 'LSTM':
          decoder_output, decoder_hidden, decoder_attention = self.decoder.forward(decoder_input, decoder_hidden, encoder_outputs)
        else:
          decoder_output, decoder_hidden, decoder_c, decoder_attention = self.decoder.forward(decoder_input, decoder_hidden, encoder_outputs, decoder_c)

        value, index = decoder_output.data.topk(1)
        decoder_input = index.squeeze().detach()
        outputs.append(decoder_input.item())
        if decoder_input.item() == end_token:
          break

      return outputs


  def evaluate(self, data):
    correct = 0
    character_wise = 0
    count = 0
    total_distance = 0

    for pair in data:
      input = pair[0]
      target = pair[1]
      pred = self.predict(input, target)
      target = target.tolist()
      target = [t[0] for t in target]

      if count%500 == 0:
        print(decoded_word(tamil,pred), decoded_word(tamil,target))
      count = count + 1

      pred_word = decoded_word(tamil,pred)
      tar_word = decoded_word(tamil,target)
      
      if pred_word == tar_word:
        correct = correct + 1
      
      total_distance = total_distance + min((Levenshtein.distance(pred_word, tar_word)/max(len(tar_word),len(pred_word))), 1)
    
    avg_distance = total_distance/len(data)
    char_acc = 1 - avg_distance
    acc = correct/len(data)


    return acc, char_acc

In [54]:
model = seq2seq_attn(inp_language = english, out_language = tamil, embedding_size = 128, hl_size = 128)

In [55]:
train_loss = 0
for i in range(0, n_iters):
  training_pair = training_pairs[i]
  x = training_pair[0]
  y = training_pair[1]
  loss = model.train_step(x, y)
  train_loss = train_loss + loss

  if (i+1)%5000 == 0:
    print('------------------------------------------------')
    print('train loss is:', train_loss/5000)
    test_acc, char_acc = model.evaluate(val_data)
    print(f'test accuracy is {test_acc} and character-wise accuracy is {char_acc}')
    train_loss = 0

------------------------------------------------
train loss is: 2.5637444081916625
சார்கககக்ககக்கக்கும் நாணயக்குற்றிகளும்
குத்தத்் கௌரவத்தை
பி்்்்் போயிங்
சார்கில் அடிகளை
பெட்டும் பிஸினஸ்
கு்திர்ககக்கு கோட்பாடுகளில்
குக்க்் கோவலர்
பி்்்்் தாலி
பெர்்் வண
test accuracy is 0.0 and character-wise accuracy is 0.26914009182847065
------------------------------------------------
train loss is: 1.872706409942668
நனனாக்குக்கிறும்முமமும் நாணயக்குற்றிகளும்
கொரவவத்தால் கௌரவத்தை
போயிக் போயிங்
அடகளிலா அடிகளை
புசிந்செ பிஸினஸ்
கொட்பாக்களில் கோட்பாடுகளில்
கொலல்ரா கோவலர்
அலின் தாலி
சென் வண
test accuracy is 0.050537109375 and character-wise accuracy is 0.612077025535356
------------------------------------------------
train loss is: 1.1699905734935776
நானயக்குட்றிில்் நாணயக்குற்றிகளும்
கொர்வாத்தால் கௌரவத்தை
போயிங்க போயிங்
அடிகளை அடிகளை
புசினெச்ச் பிஸினஸ்
கொட்பாடுகளளில் கோட்பாடுகளில்
கொவலலர் கோவலர்
டலில் தாலி
வென்்் வண
test accuracy is 0.182373046875 and character-wise accuracy is 0.7100039629199719
-----

In [56]:
import pickle
with open('lstm_attn_0.2_max_model_bidir.pkl', 'wb') as file:
    pickle.dump(model, file)
    print('Object successfully saved.')

Object successfully saved.


#Setting up wandb and hyperparameter sweep


In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 28.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.8 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=ea4c53739cb1452b6f5eff770204e537e60d97c7f872c06acb8cf68f77813a1e
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [ ]:
import wandb

In [ ]:
!wandb login
#63a3fc8a45a3ca461290e711c17782295dfe1975

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
entity_name = "clroymustang"
project_name = "cs6910-assignment-3"

In [ ]:
n_iters = 75000

In [ ]:
training_pairs = [random.choice(train_data) for i in range(0, n_iters)]

In [ ]:
def sweep():
  defaults = {
      'embedding_size': 32,
      'n_layers': 1,
      'hl_size': 32,
      'bidir': False,
      'dropout': 0,
      'cell_type': 'GRU'
  }
  wandb.init(config = defaults)
  config = wandb.config

  vanilla_model = seq2seq_vanilla(english, tamil, embedding_size = config.embedding_size, n_layers = config.n_layers, hl_size = config.hl_size, dropout = config.dropout, \
                          cell_type = config.cell_type, lr = 0.01, teacher_forcing_ratio = 0.5, bidirectional_flag = config.bidir)
  
  train_loss = 0
  for i in range(0, n_iters):
    training_pair = training_pairs[i]
    x = training_pair[0]
    y = training_pair[1]
    loss = vanilla_model.train_step(x, y)
    train_loss = train_loss + loss

    if (i+1)%5000 == 0:
      test_acc, char_acc = vanilla_model.evaluate(val_data)
      train_loss = train_loss/5000
      log_dict = {"train_loss":train_loss, "test_accuracy":test_acc, "test_similarity":char_acc}
      wandb.log(log_dict)
      train_loss = 0
  
  sweep_name = str(config.cell_type) + "_emb" + str(config.embedding_size) + "_hl" + str(config.hl_size) + "_nl" + str(config.n_layers) + "_d" + \
  str(config.dropout) + "_bidir" + str(config.bidir)
  wandb.run.name = sweep_name

  wandb.run.save()
  wandb.run.finish()


In [ ]:
hyperparameters = {
      "embedding_size": {
          'values': [32, 64, 128]
      },
      "n_layers": {
          'values' : [1, 2, 3]
      },
      "hl_size": {
          'values' : [32, 64, 128]
      },
      "bidir": {
          'values': [True, False]
      },
      "dropout":{
        'values': [0, 0.2, 0.3]
      },
      "cell_type": {
          'values' : ['RNN', 'GRU', 'LSTM']
      }
  }

In [ ]:
sweep_configuration = {
    'method': 'bayes',
    'name': 'vanilla_sweep',
    'metric': {
        'goal': 'maximize', 
        'name': 'test_similarity'
        },
    'parameters': hyperparameters
}

In [ ]:
#sweep_id = wandb.sweep(sweep_configuration, entity=entity_name, project=project_name)
sweep_id = 'td3auwvp'

In [ ]:
wandb.agent(sweep_id, sweep, project = project_name)

wandb: Agent Starting Run: 2hfvjn75 with config:
wandb: 	bidir: False
wandb: 	cell_type: LSTM
wandb: 	dropout: 0.3
wandb: 	embedding_size: 32
wandb: 	hl_size: 128
wandb: 	n_layers: 1
wandb: Currently logged in as: dhananjayb (clroymustang). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


test_accuracy,▁▁▁▁▁▂▂▃▄▅▆▆▇██
test_similarity,▁▁▂▃▃▅▅▆▇▇▇████
train_loss,█▇▇▆▆▅▄▃▃▂▂▂▁▁▁
test_accuracy,0.27515
test_similarity,0.77206
train_loss,0.74501


wandb: Agent Starting Run: 0yfik8wj with config:
wandb: 	bidir: False
wandb: 	cell_type: LSTM
wandb: 	dropout: 0
wandb: 	embedding_size: 64
wandb: 	hl_size: 128
wandb: 	n_layers: 3


test_accuracy,▁▁▁▁▁▁▂▃▄▅▆▆▇██
test_similarity,▁▁▂▂▃▄▅▆▇▇▇████
train_loss,█▇▇▆▆▅▄▃▃▂▂▂▁▁▁
test_accuracy,0.38916
test_similarity,0.83038
train_loss,0.53054


...

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


test_accuracy,▁▁▁▁▁▁▂▂▄▅▆▆▇██
test_similarity,▁▂▂▂▃▄▅▆▆▇▇████
train_loss,█▇▇▇▆▅▄▄▃▃▂▂▁▁▁
test_accuracy,0.28247
test_similarity,0.77981
train_loss,0.72771


wandb: Agent Starting Run: 1lzq4zym with config:
wandb: 	bidir: False
wandb: 	cell_type: LSTM
wandb: 	dropout: 0.2
wandb: 	embedding_size: 32
wandb: 	hl_size: 128
wandb: 	n_layers: 1


test_accuracy,▁▁▁▁▁▁▂▂▄▄▅▆▇██
test_similarity,▁▂▃▃▄▅▅▆▆▇▇▇███
train_loss,█▇▇▆▆▅▄▄▃▃▂▂▂▁▁
test_accuracy,0.30957
test_similarity,0.78346
train_loss,0.74343


wandb: Sweep Agent: Waiting for job.


tinkering

In [ ]:
def evaluate_with_beam_search(self, data, beam_size = 3):
      for pair in data:
        input = pair[0]
        target = pair[1]

        with torch.no_grad():
          encoder_hidden = self.encoder.init_hidden()
          encoder_c = self.encoder.init_hidden()

          if self.bidir == True:
            encoder_outputs = torch.zeros(self.max_length, 2*self.encoder.hl_size, device=device)
          else:
            encoder_outputs = torch.zeros(self.max_length, self.encoder.hl_size, device=device)

          input_length = input.size(0)
          for i in range(0, min(input_length, self.max_length)):
            if self.cell_type != 'LSTM':
              encoder_output, encoder_hidden = self.encoder.forward(input[i], encoder_hidden)
              encoder_outputs[i] = encoder_output[0, 0]
            else:
              encoder_output, encoder_hidden, encoder_c = self.encoder.forward(input[i], encoder_hidden, encoder_c)
              encoder_outputs[i] = encoder_output[0, 0]

          decoder_input = torch.tensor([[start_token]], device=device)
          decoder_hidden = encoder_hidden
          decoder_c = encoder_c

          possible_outputs = []
          possible_hiddens = []
          possible_probabilities = []

          while True:
            if len(possible_outputs) == 0:
              if self.cell_type != 'LSTM':
                decoder_output, decoder_hidden, decoder_attention = self.decoder.forward(decoder_input, decoder_hidden, encoder_outputs)
              else:
                decoder_output, decoder_hidden, decoder_c, decoder_attention = self.decoder.forward(decoder_input, decoder_hidden, encoder_outputs, decoder_c)
              
              values, indices = decoder_output.data.topk(beam_size)
              


          for i in range(0, self.max_length):
            if self.cell_type != 'LSTM':
              decoder_output, decoder_hidden, decoder_attention = self.decoder.forward(decoder_input, decoder_hidden, encoder_outputs)
            else:
              decoder_output, decoder_hidden, decoder_c, decoder_attention = self.decoder.forward(decoder_input, decoder_hidden, encoder_outputs, decoder_c)

            values, indices = decoder_output.data.topk(beam_size)

            decoder_input = index.squeeze().detach()
            outputs.append(decoder_input.item())
            if decoder_input.item() == end_token:
              break

          return outputs